# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Entrainement d'un modèle Word2Vec sur ce corpus (modèle 2)

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 19min 5s, sys: 47.3 s, total: 19min 52s
Wall time: 7min 21s


## Explorer le modèle

In [28]:
model = Word2Vec.load("../data/bulletins_2.model")

### Calculer la similarité entre deux termes

In [30]:
model.wv.similarity("boucher", "boulanger")

0.86908996

In [ ]:
model.wv.similarity("homme", "femme")

In [ ]:
model.wv.similarity("voiture", "carrosse")

### Chercher les mots les plus proches d'un terme donné

In [31]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8239486217498779),
 ('considere_comme_officiel', 0.7925914525985718),
 ('liege', 0.7702558040618896),
 ('paris', 0.7679300308227539),
 ('nivelles', 0.7521665096282959),
 ('gand', 0.7397889494895935),
 ('lille', 0.7102484703063965),
 ('differents_quartiers', 0.6752375960350037),
 ('lille_1860', 0.6710933446884155),
 ('louvain', 0.6700588464736938)]

In [ ]:
model.wv.most_similar("boucher", topn=10)

### Faire des recherches complexes à travers l'espace vectoriel

In [32]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.8239251375198364), ('ostende', 0.8021267056465149), ('gand', 0.7774108052253723), ('manchester', 0.7756398320198059), ('londres', 0.7693049311637878), ('tournai', 0.7622190117835999), ('verviers', 0.7603538036346436), ('paris', 0.7556376457214355), ('louvain', 0.7518820762634277), ('prague', 0.7503906488418579)]


### Modèle 3: Taille du vecteur = 100 (vs 32)

In [33]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26min 12s, sys: 1min 6s, total: 27min 19s
Wall time: 9min 32s


In [35]:
model = Word2Vec.load("../data/bulletins_3.model")

In [37]:
model.wv.similarity("boucher", "boulanger")

0.7806577

In [ ]:
model.wv.similarity("homme", "femme")

In [ ]:
model.wv.similarity("voiture", "carrosse")

In [38]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.6470252275466919),
 ('paris', 0.6332045793533325),
 ('liege', 0.631273090839386),
 ('nivelles', 0.6280113458633423),
 ('gand', 0.6024729013442993),
 ('laeken', 0.5872405767440796),
 ('lille', 0.5851026773452759),
 ('bruxelle', 0.5699034929275513),
 ('jumeaux_naissances', 0.5477592945098877),
 ('malines', 0.5462380647659302)]

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [39]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.6613479256629944), ('paris', 0.6434320211410522), ('verviers', 0.6389840841293335), ('londres', 0.6370319724082947), ('hambourg', 0.6282646059989929), ('prague', 0.6222774386405945), ('gand', 0.6142219305038452), ('tournai', 0.6134740114212036), ('liege', 0.6133009791374207), ('ostende', 0.6108174920082092)]


### Modèle 4: Vecteur 100 + élargissement de la fenêtre à 20 (vs 5)

In [40]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 31min 2s, sys: 1min 10s, total: 32min 12s
Wall time: 10min 3s


In [117]:
model = Word2Vec.load("../data/bulletins_4.model")

In [119]:
model.wv.similarity("boucher", "boulanger")

0.69779754

In [ ]:
model.wv.similarity("homme", "femme")

In [120]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6508788466453552),
 ('dette_active', 0.6217373609542847),
 ('bruxelle', 0.5955502390861511),
 ('liege', 0.586171567440033),
 ('laeken', 0.5658113956451416),
 ('nivelles', 0.559372067451477),
 ('hutel', 0.5444490909576416),
 ('vhotel', 0.5254800915718079),
 ('manufacturiere', 0.5184491276741028),
 ('bruxel', 0.5167596340179443)]

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [121]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.6075229048728943), ('manchester', 0.6038661599159241), ('hambourg', 0.5692122578620911), ('ostende', 0.5674587488174438), ('paris', 0.5573484897613525), ('lille', 0.5570899844169617), ('cologne', 0.5560243129730225), ('gand', 0.5556208491325378), ('amsterdam', 0.5449244976043701), ('tournai', 0.5420806407928467)]


### Modèle 5: Taille fenêtre 20 (vs 5) et vecteur 32 (vs 100)

In [47]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26min 43s, sys: 1min 14s, total: 27min 58s
Wall time: 9min 51s


In [49]:
model = Word2Vec.load("../data/bulletins_5.model")

In [51]:
model.wv.similarity("boucher", "boulanger")

0.8505966

In [ ]:
model.wv.similarity("homme", "femme")

In [52]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8055386543273926),
 ('liege', 0.7914955615997314),
 ('gand', 0.7288044095039368),
 ('bruxelloise', 0.7005123496055603),
 ('aux_confins', 0.6939319372177124),
 ('nivelles', 0.6921486854553223),
 ('obbgations_tramways', 0.6838479042053223),
 ('ostende', 0.6827777624130249),
 ('paris', 0.6722395420074463),
 ('louvain', 0.6675084829330444)]

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [53]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.8014519214630127), ('huy', 0.7924149632453918), ('gand', 0.7921144962310791), ('ostende', 0.7839378714561462), ('hambourg', 0.7556935548782349), ('manchester', 0.7534753680229187), ('lille', 0.7480643391609192), ('verviers', 0.7413298487663269), ('jadis', 0.7364925146102905), ('berlin', 0.735415518283844)]


### Modèle 6 : Vecteur 300 et fenêtre 10

In [54]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 52min 11s, sys: 2min 14s, total: 54min 25s
Wall time: 16min 7s


In [56]:
model = Word2Vec.load("../data/bulletins_6.model")

In [ ]:
model.wv.similarity("boucher", "boulanger")

In [ ]:
model.wv.similarity("homme", "femme")

In [95]:
model.wv.similarity("voiture", "carrosse")

0.24132355

In [ ]:
model.wv.most_similar("bruxelles", topn=10)

In [101]:
model.wv.most_similar("boucher", topn=10)

[('tailleur', 0.6623472571372986),
 ('boulanger', 0.6560649275779724),
 ('cabaretier', 0.6487619280815125),
 ('menuisier', 0.6416441798210144),
 ('serrurier', 0.6407414078712463),
 ('charcutier', 0.6258613467216492),
 ('tapissier', 0.614556074142456),
 ('terrassier', 0.6109156012535095),
 ('cordonnier', 0.6100437641143799),
 ('fabricant', 0.6039721965789795)]

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

### Modèle 7 : 300 dimensions et élargissement de la fenêtre à 40

In [140]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=40, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 1h 18min 26s, sys: 3min 23s, total: 1h 21min 49s
Wall time: 23min 12s


In [142]:
model = Word2Vec.load("../data/bulletins_7.model")

In [144]:
model.wv.similarity("boucher", "boulanger")

0.59319586

In [ ]:
model.wv.similarity("homme", "femme")

In [145]:
model.wv.similarity("voiture", "carrosse")

0.1269484

In [146]:
model.wv.most_similar("bruxelles", topn=10)

[('liege', 0.5068586468696594),
 ('nivelles', 0.49116843938827515),
 ('bruxelle', 0.4783177375793457),
 ('xelles', 0.4692355692386627),
 ('tournai', 0.4536599814891815),
 ('lille', 0.44403088092803955),
 ('ostende', 0.44255688786506653),
 ('liruxelles', 0.4339611530303955),
 ('verviers', 0.43019574880599976),
 ('dette_active', 0.4301547706127167)]

In [147]:
model.wv.most_similar("boucher", topn=10)

[('boulanger', 0.5931957364082336),
 ('marchand', 0.579157829284668),
 ('coiffeur', 0.5705497860908508),
 ('etre_abattus', 0.5697841644287109),
 ('fabricant', 0.5636791586875916),
 ('mecanicien', 0.5634440779685974),
 ('menuisier', 0.5607987642288208),
 ('cordonnier', 0.5579670071601868),
 ('serrurier', 0.5513495802879333),
 ('cabaretier', 0.5434521436691284)]

In [148]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.5512763261795044), ('lille', 0.5193424820899963), ('tournai', 0.5141823291778564), ('ostende', 0.5015013217926025), ('huy', 0.48177802562713623), ('prague', 0.4748130738735199), ('gand', 0.47399699687957764), ('lyon', 0.46861395239830017), ('rotterdam', 0.4614550471305847), ('geneve', 0.45686960220336914)]


### Modèle 8 : Vecteur 300, fenêtre 5

In [153]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 52min 31s, sys: 2min 45s, total: 55min 16s
Wall time: 17min 52s


In [155]:
model = Word2Vec.load("../data/bulletins_8.model")

In [157]:
model.wv.similarity("boucher", "boulanger")

0.6921761

In [ ]:
model.wv.similarity("homme", "femme")

In [158]:
model.wv.similarity("voiture", "carrosse")

0.20062777

In [159]:
model.wv.most_similar("bruxelles", topn=10)

[('nivelles', 0.5794664025306702),
 ('rruxelles', 0.5737338662147522),
 ('liege', 0.5225680470466614),
 ('gand', 0.5023678541183472),
 ('paris', 0.48170360922813416),
 ('xelles', 0.4629574120044708),
 ('malines', 0.4540492296218872),
 ('londres', 0.450400710105896),
 ('lille', 0.449739545583725),
 ('verviers', 0.44773247838020325)]

In [160]:
model.wv.most_similar("boucher", topn=10)

[('menuisier', 0.705475926399231),
 ('tailleur', 0.7049712538719177),
 ('cordonnier', 0.6925906538963318),
 ('boulanger', 0.6921761631965637),
 ('charcutier', 0.6875828504562378),
 ('tapissier', 0.6866316795349121),
 ('serrurier', 0.67592453956604),
 ('marbrier', 0.657673180103302),
 ('cabaretier', 0.6564407348632812),
 ('batelier', 0.6554194092750549)]

In [161]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('verviers', 0.534920334815979), ('londres', 0.5326730608940125), ('liege', 0.5236067175865173), ('tournai', 0.5141814947128296), ('gand', 0.5092669129371643), ('hambourg', 0.4946388006210327), ('nivelles', 0.4940313696861267), ('paris', 0.4934212565422058), ('prague', 0.4888829290866852), ('lille', 0.4820585250854492)]


### Modèle 9 : Vecteur 100, fenêtre 40

In [166]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=40, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 42min 18s, sys: 2min 4s, total: 44min 23s
Wall time: 14min 49s


In [168]:
model = Word2Vec.load("../data/bulletins_9.model")

In [170]:
model.wv.similarity("boucher", "boulanger")

0.68535304

In [ ]:
model.wv.similarity("homme", "femme")

In [171]:
model.wv.similarity("voiture", "carrosse")

0.12812918

In [172]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6651670336723328),
 ('liege', 0.5861473083496094),
 ('nivelles', 0.5695111155509949),
 ('laeken', 0.563617467880249),
 ('dette_active', 0.5545754432678223),
 ('bruxelle', 0.5514754056930542),
 ('bruxe', 0.540813148021698),
 ('manufacturiere', 0.5400780439376831),
 ('huy', 0.5379692316055298),
 ('ostende', 0.5323569774627686)]

In [173]:
model.wv.most_similar("boucher", topn=10)

[('cordonnier', 0.7136046290397644),
 ('fabricant', 0.695364773273468),
 ('boulanger', 0.6853529810905457),
 ('charpentier', 0.6758134365081787),
 ('marchand', 0.6743961572647095),
 ('serrurier', 0.6743265390396118),
 ('menuisier', 0.6717424988746643),
 ('charcutier', 0.6633214950561523),
 ('tailleur', 0.656404435634613),
 ('meunier', 0.6544731855392456)]

In [174]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('huy', 0.6195623278617859), ('hambourg', 0.5991153717041016), ('manchester', 0.5982712507247925), ('liege', 0.5854235887527466), ('londres', 0.5793405175209045), ('ostende', 0.5688986778259277), ('paris', 0.5649790167808533), ('lille', 0.5636627674102783), ('amsterdam', 0.5562283992767334), ('gand', 0.5400305390357971)]
